## Pre-processing

Extract all major flood dates for every country of the FEWS Network.
Extract for each event, extract food security data for before & after (month).

In [20]:
import pandas as pd
import os
import timeit
os.getcwd()
start = timeit.default_timer()


In [21]:
# import geopandas as gpd
#
# shape = gpd.read_file('flood/wlf_nhr_fl_dfomasterlist_20190418.shp')
# shape
# l = []
# for i in shape.index:
#     if shape['Country__c'][i] == 'Chad':
#         j = shape['Date_Began'][i]
#         l.append(j)

In [22]:
# Read Food Security data
FS = pd.read_csv('Data/predicting_food_crises_data.csv')


In [23]:
# Read Flood Event data
FE = pd.read_csv('Data/global_flooding_events.csv')


In [24]:
# CLEAN FE DATA
# Remove FE dates before 2010
FE.drop(FE.index[FE['year'] == 2000], inplace=True)
FE.drop(FE.index[FE['year'] == 2001], inplace=True)
FE.drop(FE.index[FE['year'] == 2002], inplace=True)
FE.drop(FE.index[FE['year'] == 2003], inplace=True)
FE.drop(FE.index[FE['year'] == 2004], inplace=True)
FE.drop(FE.index[FE['year'] == 2005], inplace=True)
FE.drop(FE.index[FE['year'] == 2006], inplace=True)
FE.drop(FE.index[FE['year'] == 2007], inplace=True)
FE.drop(FE.index[FE['year'] == 2008], inplace=True)
FE.drop(FE.index[FE['year'] == 2009], inplace=True)
FE.drop(FE.index[FE['year'] == 2010], inplace=True)
FE.drop(FE.index[FE['year'] == 2011], inplace=True)
FE.drop(FE.index[FE['year'] == 2012], inplace=True)
FE.drop(FE.index[FE['year'] == 2013], inplace=True)
FE.drop(FE.index[FE['year'] == 2014], inplace=True)

# Remove duplicates
FE = FE.drop_duplicates(subset=['country', 'year', 'month'])
### Try to group duplicates by average dates

In [25]:
# Extract Flood events data that only includes FEW NET countries
country_list = set(FS['country'].tolist()) # Extract list of FEWS NET countries
FE2 = pd.DataFrame(columns=list(FE.columns))
for i in FE.index:
    for country in country_list:
        if FE['country'][i] == country:
            FE2 = FE2.append(FE.loc[i])
FE2 = FE2.sort_values(by=['country', 'year', 'month', 'day'])
FE2 = FE2.drop(['index', 'area', 'exposed'], axis=1)
country_list

{'Afghanistan',
 'Burkina Faso',
 'Chad',
 'Democratic Republic of Congo',
 'Ethiopia',
 'Guatemala',
 'Haiti',
 'Kenya',
 'Malawi',
 'Mali',
 'Mauritania',
 'Mozambique',
 'Niger',
 'Nigeria',
 'Somalia',
 'South Sudan',
 'Sudan',
 'Uganda',
 'Yemen',
 'Zambia',
 'Zimbabwe'}

In [26]:
# Create Timestamps (MONTHLY)
# FS
FS['date'] = FS['year'].astype(str) + '-' + FS['month'].astype(str)
FS['datetime'] = pd.to_datetime(FS['date'])
FS = FS.drop(['date'], axis=1)
#FE
FE2['date'] = FE2['year'].astype(str) + '-' + FE2['month'].astype(str) + '-' +FE2['day'].astype(str)
FE2['datetime'] = pd.to_datetime(FE2['date'])
FE2 = FE2.drop(['date'], axis=1)
FE2['datetime_round'] = pd.to_datetime(FE2['datetime']).dt.to_period('M').dt.to_timestamp()
FE2 = FE2.reset_index()
FE2 = FE2.drop(['index'], axis=1)

In [27]:
### SELECT COUNTRY OF INTEREST
country = 'Mozambique'
FS.drop(FS.loc[FS['country'] != country].index, inplace=True)
FE2.drop(FE2.loc[FE2['country'] != country].index, inplace=True)
# Reset index
FS = FS.reset_index()
FS = FS.drop(['index'], axis=1)
FE2 = FE2.reset_index()
FE2 = FE2.drop(['index'], axis=1)

# FE2

Something is wrong with the first event in Afghanistan dataset (code works for Burkina faso) - code needs error handling

In [28]:
# # Filter FS data to only include data before/after/during each event
# FS2 = pd.DataFrame(columns=list(FS.columns))
# FS2['State'] = ''
# FS2['Flood Date'] = ''
# for event in FE2.index:
#     for i in FS.index:
#         if FS['country'][i] == FE2['country'][event]:
#             FS_date = FS['datetime'][i]
#             FE_date = FE2['datetime_round'][event]
#             before = FE_date - pd.DateOffset(months=1)
#             after = FE_date + pd.DateOffset(months=1)
#             if FS_date == before:
#                 FS2 = FS2.append(FS.loc[i])
#                 FS2.loc[i,['State']] = 'Before'
#                 FS2.loc[i, ['Flood Date']] = FE2['datetime'][event]
#             elif FS_date == after:
#                 FS2 = FS2.append(FS.loc[i])
#                 FS2.loc[i,['State']] = 'After'
#                 FS2.loc[i, ['Flood Date']] = FE2['datetime'][event]
#             else:
#                 continue


In [29]:
FS2 = FS2.reset_index()
FS2 = FS2.drop(['index'], axis=1)
# FS2['datetime'] = FS2['datetime'].astype(str)
# FS2['Flood Date'] = FS2['Flood Date'].astype(str)
# FS2 = FS2[['country', 'admin_code', 'admin_name', 'year_month', 'year', 'month', 'Flood Date', 'State', 'datetime',
#            'fews_ipc', 'ndvi_mean', 'rain_mean', 'et_mean', 'acled_coun', 'acled_fata', 'p_staple_f', 'area',
#            'cropland_p', 'pop', 'ruggedness', 'pasture_pc', 'spacelag', 'timelag1', 'timelag2', 'geometry']]

# For GEE - before/after dates
FE3 = FS2[['country', 'year', 'month', 'Flood Date', 'datetime', 'State']]
FE3 = FE3.drop_duplicates()
FE3 = FE3.reset_index()
FE3 = FE3.drop(['index'], axis=1)

# For GEE - flood dates only
# FE4 = FE2[['datetime']]
# FE4 = FE4.reset_index()
# FE4 = FE4.drop(['index'], axis=1)

stop = timeit.default_timer()
print('Running Time: ', stop - start, 'seconds')

NameError: name 'FS2' is not defined

In [ ]:
# Save data
# FS2.to_csv('Data/FE_before_after_data.csv', index = False)
# FE3.to_csv('Data/FE_before_after_dates.csv', index = False)
# FE4.to_csv('Data/GEE_flood_dates.csv', index = False)
# FS2.to_file('Data/FS_before_after_data.shp')
